# Predicting Apple Stock trend with Gradient Booster Model

Gradient Booster Algorithm is used to identify trend in Apple stock. Using the `APPL.csv` file.

In [2]:
# Initial imports
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import GradientBoostingClassifier
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Loading and Preprocessing Data

Load the `APPL.csv` in a pandas DataFrame called `df`

In [3]:
# Loading data
file_path = Path("AAPL.csv")
df = pd.read_csv(file_path)
df.dropna(inplace=True)
df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,ts_polarity,twitter_volume
917,2019-08-26,51.47,51.80,51.26,51.62,51.12,104174400,0.072340,888.0
918,2019-08-27,51.97,52.14,50.88,51.04,50.54,103493200,0.117541,962.0
919,2019-08-28,51.03,51.43,50.83,51.38,50.88,63755200,0.061477,895.0
920,2019-08-29,52.13,52.33,51.67,52.25,51.74,83962000,0.056460,1083.0
921,2019-08-30,52.54,52.61,51.80,52.19,51.67,84573600,0.106096,1005.0


In [4]:
#OHLC Chart with Volume
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Ohlc(x=df.Date,
                      open=df.Open,
                      high=df.High,
                      low=df.Low,
                      close=df.Close,
                      name='Price'), row=1, col=1)

fig.add_trace(go.Scatter(x=df.Date, y=df.Volume, name='Volume'), row=2, col=1)

fig.update_layout(title_text="Apple Stock Price and Volume")
fig.update(layout_xaxis_rangeslider_visible=False)
fig.show()

In [5]:
# Dataframe with Date, Adj close,  Volume, ts_polarity, twitter_volume of APPL
appl_df = df[["Date", "Adj Close", "Volume", "ts_polarity", "twitter_volume"]]
appl_df.head()

,Date,Adj Close,Volume,ts_polarity,twitter_volume
0,2016-01-04,24.44,270597600,0.070389,1133.0
1,2016-01-05,23.83,223164000,0.133635,1430.0
2,2016-01-06,23.36,273829600,0.072042,1949.0
3,2016-01-07,22.38,324377600,0.074369,2289.0
4,2016-01-08,22.50,283192000,0.051595,2235.0


In [6]:
# Setting Index as Date
appl_df = appl_df.set_index("Date")
appl_df.tail()

,Adj Close,Volume,ts_polarity,twitter_volume
Date,,,,
2019-08-26,51.12,104174400,0.072340,888.0
2019-08-27,50.54,103493200,0.117541,962.0
2019-08-28,50.88,63755200,0.061477,895.0
2019-08-29,51.74,83962000,0.056460,1083.0
2019-08-30,51.67,84573600,0.106096,1005.0


In [7]:
# Sorting ts_polarity into Positive, Negative and Neutral sentiment

sentiment = []
for score in appl_df['ts_polarity']:
    if score >= 0.05 :
          sentiment.append("Positive")
    elif score <= - 0.05 :
          sentiment.append("Negative")
    else :
        sentiment.append("Neutral")

appl_df["Sentiment"] = sentiment
appl_df.head()

,Adj Close,Volume,ts_polarity,twitter_volume,Sentiment
Date,,,,,
2016-01-04,24.44,270597600,0.070389,1133.0,Positive
2016-01-05,23.83,223164000,0.133635,1430.0,Positive
2016-01-06,23.36,273829600,0.072042,1949.0,Positive
2016-01-07,22.38,324377600,0.074369,2289.0,Positive
2016-01-08,22.50,283192000,0.051595,2235.0,Positive


In [8]:
# Sentiment Count
appl_df['Sentiment'].value_counts()

Positive    785
Neutral     134
Negative      3
Name: Sentiment, dtype: int64

In [9]:
#Stock Trend based on difference between current price to previous day price and coverting them to '0' as fall and '1' as rise in stock price
appl_df['Price Diff'] = appl_df['Adj Close'].diff()
appl_df.dropna(inplace = True)
appl_df['Trend'] = np.where(
    appl_df['Price Diff'] > 0 , 1, 0)

appl_df.head()

,Adj Close,Volume,ts_polarity,twitter_volume,Sentiment,Price Diff,Trend
Date,,,,,,,
2016-01-05,23.83,223164000,0.133635,1430.0,Positive,-0.61,0
2016-01-06,23.36,273829600,0.072042,1949.0,Positive,-0.47,0
2016-01-07,22.38,324377600,0.074369,2289.0,Positive,-0.98,0
2016-01-08,22.50,283192000,0.051595,2235.0,Positive,0.12,1
2016-01-11,22.86,198957600,0.019443,1222.0,Neutral,0.36,1


In [10]:
# Binary encoding Sentiment column
appl_trend = appl_df[["Adj Close", "Volume", 'twitter_volume', "Sentiment", "Trend"]]
appl_trend = pd.get_dummies(appl_trend, columns=["Sentiment"])
appl_trend.head()

,Adj Close,Volume,twitter_volume,Trend,Sentiment_Negative,Sentiment_Neutral,Sentiment_Positive
Date,,,,,,,
2016-01-05,23.83,223164000,1430.0,0,0,0,1
2016-01-06,23.36,273829600,1949.0,0,0,0,1
2016-01-07,22.38,324377600,2289.0,0,0,0,1
2016-01-08,22.50,283192000,2235.0,1,0,0,1
2016-01-11,22.86,198957600,1222.0,1,0,1,0


In [11]:
# Defining features set
X = appl_trend.copy()
X.drop("Trend", axis=1, inplace=True)
X.head()


,Adj Close,Volume,twitter_volume,Sentiment_Negative,Sentiment_Neutral,Sentiment_Positive
Date,,,,,,
2016-01-05,23.83,223164000,1430.0,0,0,1
2016-01-06,23.36,273829600,1949.0,0,0,1
2016-01-07,22.38,324377600,2289.0,0,0,1
2016-01-08,22.50,283192000,2235.0,0,0,1
2016-01-11,22.86,198957600,1222.0,0,1,0


In [12]:
# Defining target vector
y = appl_trend["Trend"].values.reshape(-1, 1)
y[:5]


array([[0],
       [0],
       [0],
       [1],
       [1]])

Split the data into training and testing sets.

In [13]:
# Splitting into Train and Test sets
split = int(0.7 * len(X))

X_train = X[: split]
X_test = X[split:]

y_train = y[: split]
y_test = y[split:]

In [14]:
# Using StandardScaler to scale features data
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

 ## Create a Gradient Booster Model


In [15]:
# Choosing learning rate
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    model = GradientBoostingClassifier(n_estimators=20,
                                      learning_rate=learning_rate,
                                      max_features=2,
                                      max_depth=3,
                                      random_state=0)
    model.fit(X_train_scaled,y_train.ravel())
    print("Learning rate: ", learning_rate)

    # Scoring the model
    print("Accuracy score (training): {0:.3f}".format(
        model.score(
            X_train_scaled,
            y_train.ravel())))
    print("Accuracy score (validation): {0:.3f}".format(
        model.score(
            X_test_scaled,
            y_test.ravel())))
    print()

Learning rate:  0.05
Accuracy score (training): 0.602
Accuracy score (validation): 0.549

Learning rate:  0.1
Accuracy score (training): 0.651
Accuracy score (validation): 0.570

Learning rate:  0.25
Accuracy score (training): 0.693
Accuracy score (validation): 0.563

Learning rate:  0.5
Accuracy score (training): 0.719
Accuracy score (validation): 0.534

Learning rate:  0.75
Accuracy score (training): 0.756
Accuracy score (validation): 0.563

Learning rate:  1
Accuracy score (training): 0.738
Accuracy score (validation): 0.538



In [16]:
# Creating GradientBoostingClassifier model
classifier = GradientBoostingClassifier(n_estimators=20,
                                        learning_rate=0.75,
                                        max_features=5,
                                        max_depth=3,
                                        random_state=0)

# Fitting the model
classifier.fit(X_train_scaled, y_train.ravel())
# Scoring the model
print("Accuracy score (training): {0:.3f}".format(
    model.score(
        X_train_scaled,
        y_train)))
print("Accuracy score (validation): {0:.3f}".format(
    model.score(
        X_test_scaled,
        y_test)))

Accuracy score (training): 0.738
Accuracy score (validation): 0.538


## Making Predictions Using the Gradient Booster Model

In [17]:
# Making predictions
predictions = classifier.predict(X_test_scaled)
pd.DataFrame({"Prediction": predictions, "Actual": y_test.ravel()}).head(20)

# Generating accuracy score for predictions using y_test
acc_score = accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score}")

Accuracy Score : 0.5487364620938628


## Model Evaluation

Evaluating model's results, using `sklearn` to calculate the confusion matrix and to generate the classification report.


In [18]:
# Generating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"],
    columns=["Predicted 0", "Predicted 1"]
)

# Displaying results
display(cm_df)

,Predicted 0,Predicted 1
Actual 0,27,100
Actual 1,25,125


In [19]:
# Generating classification report
print("Classification Report")
print(classification_report(y_test, predictions))


Classification Report
              precision    recall  f1-score   support

           0       0.52      0.21      0.30       127
           1       0.56      0.83      0.67       150

    accuracy                           0.55       277
   macro avg       0.54      0.52      0.48       277
weighted avg       0.54      0.55      0.50       277

